In [1]:
import numpy as np
import galois
from itertools import islice
from quaos.core.circuits.utils import transvection, transvection_matrix, symplectic_product
from quaos.core.circuits.find_symplectic import check_mappable_via_clifford, solve_gf2
from quaos.core.circuits.find_symplectic_qudits import *
from quaos.utils import get_linear_dependencies
from quaos.models import random_hamiltonian
from quaos.core.circuits import  Circuit

In [7]:
n=10
for p in range(2, 20):
    for i in range(10000):
        input_ps = np.random.randint(p, size=2 * n)
        output_ps= np.random.randint(p, size=2 * n)
        if galois.is_prime(p) and not np.all(input_ps == 0) and not np.all(output_ps == 0):
            F_h= Find_transvection_map(input_ps, output_ps, p)
            if (input_ps @ F_h %p != output_ps).all():
                raise ValueError("Failed to construct valid transvection map: for u->v")

In [8]:
for p in range(2, 100):
    if galois.is_prime(p):
        for pi1 in range(1, p):
            x= 2*pi1 % p
            for pi in range(1, p):
                piinv= modinv(pi, p)
                if piinv*x % p == 0:

                    print(p,pi1, piinv, )
        print()

2 1 1



























In [9]:
n=2
for p in range(2, 20):
    if galois.is_prime(p):
        for i in range(1000):
            input_ps = np.random.randint(p, size=2 * n)
            output_ps= np.random.randint(p, size=2 * n)
            add_inv_input_ps= -input_ps % p
            assert (symplectic_product(input_ps, output_ps, p) + symplectic_product(add_inv_input_ps, output_ps, p)) % p== 0


In [10]:
n = 2 # , 50)  # Number of qudits
print(n)
allowed_dims = [3]  # , 3, 5, 7, 11]  # allowed dimensions
dimensions = [3]*n  # dimensions
for _ in range(n):
    dimensions.append(int(np.random.choice(allowed_dims)))
m = 2
pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
print(pl_sum.tableau())
print()
basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
#print("banana")
pl_sum = pl_sum[basis_indices]
print(pl_sum.tableau())
print()
# scramble input hamiltonian to get target
C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
target_pl_sum = C.act(pl_sum)
# target hamiltonian
sym_sum = pl_sum.tableau()
target_sym_sum = target_pl_sum.tableau()

print()
#print(pl_sum)
#print(pl_sum.symplectic_product_matrix())
print()
print(target_pl_sum.tableau())
print()


2
[[2 2 1 2 0 2 0 0]
 [2 1 0 0 1 2 1 2]
 [1 2 0 0 2 1 2 1]
 [1 1 2 1 0 1 0 0]]

[[2 2 1 2 0 2 0 0]
 [2 1 0 0 1 2 1 2]]



[[2 0 1 1 1 0 1 0]
 [1 2 1 1 0 1 2 0]]



In [11]:
dimensions

[3, 3, 3, 3]

In [12]:
for trial in range(100):
    n = 2 
    allowed_dims = [2]  # , 3, 5, 7, 11]  # allowed dimensions
    dimensions = []  # dimensions
    for _ in range(n):
        dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
    m = 10 #num pauli
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]
    # scramble input hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab= pl_sum.tableau()
    output_tab= target_pl_sum.tableau()
    p=2
    if check_mappable_via_clifford(input_tab, output_tab) and len(input_tab) > 1:
        F_h0= Find_transvection_map(input_tab[0], output_tab[0], p)
        assert (input_tab[0] @ F_h0 %p == output_tab[0]).all()
        u2p = input_tab[1] @ F_h0 %p
        F_h1= Find_transvection_map(u2p, output_tab[1], p)
        assert (u2p @ F_h1 %p == output_tab[1]).all()
        if symplectic_product(u2p, output_tab[1], p) !=0:
            assert (output_tab[0] @ F_h1 %p == output_tab[0]).all()
    print(trial)
       

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [13]:
print(C)

SUM [0, 1]
S [0]
H [1]
SUM [0, 1]
SUM [0, 1]
SUM [0, 1]
SUM [1, 0]
SUM [0, 1]
S [0]
SUM [0, 1]
SUM [1, 0]
H [1]
S [0]
S [0]
H [0]
H [0]
S [0]
S [0]
S [0]
SUM [1, 0]
SUM [1, 0]
SUM [1, 0]
SUM [1, 0]
H [0]
SUM [0, 1]
S [1]
H [1]
S [0]
H [1]
S [1]
H [1]
S [0]
H [0]
S [1]
H [0]
SUM [0, 1]
H [1]
H [1]
H [1]
SUM [1, 0]



In [14]:
def map_pauli_sum_to_target_tableau(pauli_sum_tableau: np.ndarray, target_pauli_sum_tableau: np.ndarray) -> np.ndarray:
    """
    Map a Pauli sum to a target Pauli sum using symplectic transvections.
    """
    if not check_mappable_via_clifford(pauli_sum_tableau, target_pauli_sum_tableau):
        raise Exception(f'Cannot map\n{pauli_sum_tableau} to\n{target_pauli_sum_tableau}')
    # Find a vector w such that <w, pauli_sum> = <w, target_pauli_sum> = 1

    m = len(pauli_sum_tableau)
    n = len(pauli_sum_tableau[0]) // 2
    mapped_paulis = []
    F = np.eye(2 * n, dtype=int)
    for i in range(m):
        # update the starting point to whatever previous solutions mapped it to
        ps = (pauli_sum_tableau[i] @ F) % 2
        target_ps = target_pauli_sum_tableau[i]

        # if np.array_equal(ps, target_ps):
        #     continue

        F_map = map_single_pauli_string_to_target(ps, target_ps, mapped_paulis)
        assert np.all((ps @ F_map) % 2 == target_ps), f"\n{F_map}\n{ps}\n{(ps @ F_map) % 2}\n{target_ps}"
        for mp in mapped_paulis:
            assert np.all((mp @ F_map) % 2 == mp), f"\n{F_map}\n{mp}\n{(mp @ F_map) % 2}"
        mapped_paulis.append(target_ps)  # these are now the constraints for the next iteration
        F = (F @ F_map) % 2

    return F



In [15]:
from quaos.core.circuits.find_symplectic import *
F=map_pauli_sum_to_target_tableau(input_tab,  output_tab)

In [16]:

for trial in range(1000):
    n = 10
    allowed_dims = [3]  # , 3, 5, 7, 11]  # allowed dimensions
    dimensions = [3]*n  # dimensions
    for _ in range(n):
        dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
    m = 10 #num pauli
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]
    # scramble input hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab= pl_sum.tableau()
    output_tab= target_pl_sum.tableau()
    store=np.zeros(len(input_tab))
    for  i in range(len(input_tab)):
        store[i]=symplectic_product(input_tab[i], output_tab[i], p)
        F=map_pauli_sum_to_target_tableau(input_tab,  output_tab) 
        mapped_tab= input_tab @ F % p
        assert (output_tab== mapped_tab).all()

 

Exception: SPM not equal. Cannot map
[[2 2 0 1 1 2 1 0 1 1 2 1 0 0 2 0 1 1 1 1 2 0 2 0 0 2 2 1 0 1 2 2 0 2 2 2
  1 1 2 0]
 [2 2 0 0 0 1 0 2 2 1 0 1 2 1 0 1 1 0 2 0 2 2 0 2 2 1 0 1 0 2 1 1 2 0 0 0
  2 2 1 0]
 [2 1 2 2 2 2 0 1 1 0 1 2 0 1 2 2 1 2 2 2 2 1 0 0 1 1 0 1 2 0 0 2 1 2 2 2
  2 2 2 0]
 [2 1 0 0 0 1 2 0 0 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 2 1 2 0 0 1 2 2 0 2 0 0
  0 2 0 1]
 [2 0 2 1 2 2 1 0 0 2 1 0 0 1 1 1 2 0 2 1 2 2 1 0 2 1 0 1 0 2 0 1 1 1 0 1
  2 2 2 2]
 [0 2 2 1 2 0 0 2 1 0 0 2 1 0 1 2 2 1 2 0 2 0 0 0 0 0 2 1 0 0 0 0 2 1 2 1
  0 2 1 0]
 [0 2 1 2 1 1 2 0 0 2 2 1 2 2 0 2 2 1 1 2 2 2 0 2 0 0 2 0 2 1 1 0 1 2 2 1
  1 2 1 1]
 [0 2 0 1 2 2 2 2 1 0 2 2 2 0 2 2 1 0 0 1 0 0 0 1 2 2 1 1 2 1 2 1 2 0 0 2
  2 1 1 2]
 [0 2 0 0 1 0 0 1 0 1 2 2 1 2 2 0 2 0 1 1 1 0 0 2 1 2 2 2 1 0 1 0 1 2 1 2
  2 1 0 1]
 [0 0 2 0 2 0 2 2 0 2 2 2 2 1 1 0 0 1 1 2 0 0 2 2 2 0 0 0 1 2 0 1 1 2 0 1
  1 2 2 1]] to
[[1 2 2 0 0 0 1 0 1 2 0 2 0 1 1 2 2 1 2 0 0 0 2 1 1 2 2 2 1 2 2 0 0 0 0 2
  2 0 2 0]
 [1 2 2 1 1 1 0 2 0 1 1 

In [17]:
n = 2
allowed_dims = [2]  # , 3, 5, 7, 11]  # allowed dimensions
dimensions = []  # dimensions
for _ in range(n):
    dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
m = 2 #num pauli
pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
pl_sum = pl_sum[basis_indices]
# scramble input hamiltonian to get target
C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
target_pl_sum = C.act(pl_sum)

input_tab= pl_sum.tableau()
output_tab= target_pl_sum.tableau()

m = len(input_tab)
n = len(input_tab[0]) // 2
mapped_paulis = []
F = np.eye(2 * n, dtype=int)
for i in range(m):
    # update the starting point to whatever previous solutions mapped it to
    ps = (input_tab[i] @ F) % 2
    target_ps = output_tab[i]

    F_map = map_single_pauli_string_to_target(ps, target_ps, mapped_paulis)
    assert np.all((ps @ F_map) % 2 == target_ps), f"\n{F_map}\n{ps}\n{(ps @ F_map) % 2}\n{target_ps}"
    for mp in mapped_paulis:
        assert np.all((mp @ F_map) % 2 == mp), f"\n{F_map}\n{mp}\n{(mp @ F_map) % 2}"
    mapped_paulis.append(target_ps)
    print(mapped_paulis)  # these are now the constraints for the next iteration
    F = (F @ F_map) % 2


[array([0, 1, 0, 0])]
[array([0, 1, 0, 0]), array([1, 1, 1, 0])]


In [ ]:
mapped_paulis

[array([0, 1, 1, 1]), array([1, 0, 1, 1])]

In [18]:
def map_single_pauli_string_to_target(pauli_string_tableau: np.ndarray, target_pauli_string_tableau: np.ndarray,
                                      constraint_paulis: list | None = None):
    sp = symplectic_product(pauli_string_tableau, target_pauli_string_tableau)
    if sp == 1:
        h = pauli_string_tableau + target_pauli_string_tableau

        F_h = transvection_matrix(h)

        return F_h

    elif sp == 0:
        w = find_symplectic_solution_extended(pauli_string_tableau, target_pauli_string_tableau, constraint_paulis)
        h_1 = target_pauli_string_tableau + w
        h_2 = pauli_string_tableau + w

        F_h_1 = transvection_matrix(h_1)
        F_h_2 = transvection_matrix(h_2)

        return (F_h_1 @ F_h_2) % 2

    else:
        raise Exception(f'sp = {sp}...This should never happen')


In [19]:
def modinv(a, p):
    """Modular inverse in GF(p)"""
    return pow(int(a), p-2, p)

def pair(vec, i, n):
    """Return 2x1 pair at index i"""
    return np.array([vec[i], vec[i+n]])

def solve_non_proportional(u_i, v_i, p):
    """Solve B x = y for non-proportional 2x2 vectors"""

    GF = galois.GF(p)
    B = np.array([[u_i[0], -u_i[1]], [-v_i[0], v_i[1]], u1i[0], -u1i[1]]) % p
    y = GF([1, 1])
    B = GF(B)
    x = np.linalg.solve(B, y).astype(np.int32).view(np.ndarray) % p
    return x

def proportionality_constant_mod(u_i, v_i, p):
    """
    Compute the proportionality constant k mod p such that v_i = k * u_i (mod p),
    assuming u_i and v_i are proportional.
    """
    a, b = u_i
    c, d = v_i

    if not galois.is_prime(p):
        raise ValueError(f"{p} is not prime!")

    # Convert to Python ints to avoid NumPy overflow
    a, b, c, d = int(a), int(b), int(c), int(d)

    # Case 1: all entries nonzero
    if a != 0 and b != 0 and c != 0 and d != 0:
        k1 = (a * pow(c, p-2, p)) % p  # k = a / c mod p
        k2 = (b * pow(d, p-2, p)) % p  # k = b / d mod p
        if k1 != k2:
            raise ValueError("Vectors are not proportional")
        return k1

    # Case 2: only second component nonzero
    elif b != 0 and d != 0:
        k = (b * pow(d, p-2, p)) % p
        return k

    # Case 3: only first component nonzero
    elif a != 0 and c != 0:
        k = (a * pow(c, p-2, p)) % p
        return k

    else:
        raise ValueError("Cannot determine proportionality constant: one of the components is zero")


def solve_proportional(u_i, v_i, p, proportionality_const):
    """Solve B x = y for proportional pair with free parameter"""

    GF = galois.GF(p)
    B = np.array([[u_i[0], -u_i[1]], [-v_i[0], v_i[1]]]) % p
    y = GF([1, proportionality_const])
    B = GF(B)
    row1 = B[0]
    t_free = proportionality_const
    solutions = []

    if row1[0] != 0 and row1[1] != 0:
        x1 = (y[0] - row1[1]*t_free) * GF(int(row1[0]))**(-1)
        solutions.append(GF([x1, t_free]))
    elif row1[0] != 0 and row1[1] == 0:
        x2 = (y[0] - row1[0]*t_free) * GF(int(row1[0]))**(-1)
        solutions.append(GF([t_free, x2]))
    elif row1[0] == 0 and row1[1] != 0:
        x2 = (y[0] - row1[0]*t_free) * GF(int(row1[1]))**(-1)
        solutions.append(GF([x2, t_free]))

    return solutions[0].astype(np.int32).view(np.ndarray)


def build_symplectic_for_transvection(u, v, p):
    """
    Build the intermediate vector w for two-step transvections if [u, v]=0 mod p 
    Assumes qudit dimension p is prime and symplectic_product(u, v, p) == 0
    """
    # --- Input validation ---
    if np.all(u == 0) or np.all(v == 0):
        raise ValueError("Input vectors cannot be zero")
    if not galois.is_prime(p):
        raise NotImplementedError(f"Prime dimension expected, got p={p}")
    if symplectic_product(u, v, p) != 0:
        raise ValueError("Symplectic product must be zero")

    n = len(u) // 2
    w = np.zeros(2*n, dtype=int)
    GF = galois.GF(p)

    # --- First pass: find non-proportional pair ---
    for i, (ui, vi) in enumerate(islice(zip(u, v), n)):
        u_i = pair(u, i, n)
        v_i = pair(v, i, n)
        nz_u, nz_v = np.count_nonzero(u_i), np.count_nonzero(v_i)
        sp_uv = symplectic_product(u_i, v_i, p)

        if nz_u and nz_v and sp_uv != 0:
            x = solve_non_proportional(u_i, v_i, p)
            w[i+n], w[i] = x[0], x[1]
            break

    # --- Second pass: proportional pair ---
    if np.all(w == 0):
        for i, (ui, vi) in enumerate(islice(zip(u, v), n)):
            u_i = pair(u, i, n)
            v_i = pair(v, i, n)
            nz_u, nz_v = np.count_nonzero(u_i), np.count_nonzero(v_i)
            sp_uv = symplectic_product(u_i, v_i, p)

            if nz_u and nz_v and sp_uv == 0:
                k = proportionality_constant_mod(u_i, v_i, p)
                x = solve_proportional(u_i, v_i, p,  k)
                w[i+n], w[i] = x[0], x[1]
                break

    # --- Third pass: handle single nonzero entries manually ---
    if np.all(w == 0):
        for i in range(n):
            u_i, v_i = pair(u, i, n), pair(v, i, n)

            # Case u_i nonzero, v_i zero
            if np.count_nonzero(u_i) != 0 and np.count_nonzero(v_i) == 0:
                w_u = np.zeros(2, dtype=int)
                if u_i[0] != 0:
                    w_u[1] = modinv(u_i[0],p)
                else:
                    w_u[0] = modinv(-u_i[1],p)
                assert symplectic_product(u_i, w_u, p) == 1
                w[i], w[i+n] = w_u[0], w_u[1]
                break

        for i in range(n):
            u_i, v_i = pair(u, i, n), pair(v, i, n)

            # Case u_i zero, v_i nonzero
            if np.count_nonzero(u_i) == 0 and np.count_nonzero(v_i) != 0:
                w_v = np.zeros(2, dtype=int)
                if v_i[0] != 0:
                    w_v[1] = (-modinv(v_i[0],p)) % p
                else:
                    w_v[0] = modinv(v_i[1],p)
                assert symplectic_product(w_v, v_i, p) == 1
                w[i], w[i+n] = w_v[0], w_v[1]
                break

    # --- Final validation ---
    if symplectic_product(u, w, p) == 0 or symplectic_product(w, v, p) == 0:
        raise ValueError("Failed to construct valid symplectic vector w")

    return w


In [20]:
def solve_non_proportional_extended(u_i, v_i, u1i, a, p):
    """Solve B x = y for non-proportional 2x2 vectors"""

    GF = galois.GF(p)
    B = np.array([[u_i[0], -u_i[1]], [-v_i[0], v_i[1]], u1i[0], -u1i[1]]) % p
    y = GF([1, 1, a])
    B = GF(B)
    x = np.linalg.solve(B, y).astype(np.int32).view(np.ndarray) % p
    return x

In [21]:
for trial in range(10):
    n = 4
    allowed_dims = [2]  # , 3, 5, 7, 11]  # allowed dimensions
    dimensions = []  # dimensions
    for _ in range(n):
        dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
    m = 2 #num pauli
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]
    # scramble input hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab= pl_sum.tableau()
    output_tab= target_pl_sum.tableau()
    u1, v1= input_tab[0], output_tab[0]
    u2, v2= input_tab[1], output_tab[1]
    F_h_1 =Find_transvection_map(u1, v1, p)
    u2p= u2 @ F_h_1 
    print(symplectic_product(u2p, v2, p))
    if symplectic_product(u2p, v2, p) == 0:
        break

0


In [22]:
def solve_non_proportional_extended(u_i, v_i, v1i, a, p):
    """Solve B x = y for non-proportional 2x2 vectors"""

    #GF = galois.GF(p)
    B = np.array([[u_i[0], -u_i[1]], [-v_i[0], v_i[1]], [u1i[0], -u1i[1]]]) % p
    y = np.array([1, 1, a])
    #B = GF(B)
    x = solve_gf2(B, y)
    return x

In [23]:
u

NameError: name 'u' is not defined

In [24]:
A_ext = np.vstack([A, [v1, v2]])
b_ext = np.append(b, a)
x = solve_gf2(A_ext, b_ext)


NameError: name 'A' is not defined

In [25]:
import numpy as np

def modinv(a, p):
    """Multiplicative inverse of a mod p (works for prime p)."""
    return pow(int(a), -1, p)

def solve_gfp(A: np.ndarray, b: np.ndarray, p: int):
    """
    Solve Ax = b over GF(p) using Gaussian elimination.
    Works for rectangular A (m x n).
    
    Args:
        A : coefficient matrix (m x n)
        b : right-hand side vector (m,)
        p : prime
    
    Returns:
        List of all solutions x in GF(p)^n (may be empty).
    """
    A = A.astype(int) % p
    b = b.astype(int) % p
    m, n = A.shape

    # Augmented matrix
    Ab = np.hstack([A, b.reshape(-1, 1)]) % p

    pivot_row = 0
    pivot_cols = []

    for col in range(n):
        # Find pivot
        pivot = None
        for row in range(pivot_row, m):
            if Ab[row, col] != 0:
                pivot = row
                break
        if pivot is None:
            continue

        # Swap into place
        if pivot != pivot_row:
            Ab[[pivot_row, pivot]] = Ab[[pivot, pivot_row]]

        # Normalize pivot row
        inv = modinv(Ab[pivot_row, col], p)
        Ab[pivot_row] = (Ab[pivot_row] * inv) % p

        # Eliminate other rows
        for row in range(m):
            if row != pivot_row and Ab[row, col] != 0:
                factor = Ab[row, col]
                Ab[row] = (Ab[row] - factor * Ab[pivot_row]) % p

        pivot_cols.append(col)
        pivot_row += 1
        if pivot_row == m:
            break

    # Check inconsistency
    for row in range(pivot_row, m):
        if Ab[row, :-1].sum() == 0 and Ab[row, -1] != 0:
            return []  # no solution

    # Identify free vars
    free_vars = [j for j in range(n) if j not in pivot_cols]

    solutions = []
    # Enumerate all free variable assignments
    for mask in np.ndindex(*(p for _ in free_vars)):
        x = np.zeros(n, dtype=int)
        # Assign free vars
        for val, col in zip(mask, free_vars):
            x[col] = val

        # Back substitution for pivot vars
        for i, col in enumerate(pivot_cols):
            row = i
            val = Ab[row, -1]
            for j in range(col + 1, n):
                val = (val - Ab[row, j] * x[j]) % p
            x[col] = val % p
        solutions.append(x)
    
   
    return solutions[0] if solutions else None



In [26]:
import numpy as np

def modinv(a, p):
    """Multiplicative inverse of a mod p (works for prime p)."""
    return pow(int(a), -1, p)

def solve_gfp(A: np.ndarray, b: np.ndarray, p: int):
    """
    Solve Ax = b over GF(p) using Gaussian elimination.
    Returns one valid solution vector x (length n), or None if no solution exists.
    
    Args:
        A : coefficient matrix (m x n)
        b : right-hand side vector (m,)
        p : prime
    
    Returns:
        x : one solution in GF(p)^n as np.ndarray, or None
    """
    A = A.astype(int) % p
    b = b.astype(int) % p
    m, n = A.shape

    # Augmented matrix
    Ab = np.hstack([A, b.reshape(-1, 1)]) % p

    pivot_row = 0
    pivot_cols = []

    for col in range(n):
        # Find pivot
        pivot = None
        for row in range(pivot_row, m):
            if Ab[row, col] != 0:
                pivot = row
                break
        if pivot is None:
            continue

        # Swap into place
        if pivot != pivot_row:
            Ab[[pivot_row, pivot]] = Ab[[pivot, pivot_row]]

        # Normalize pivot row
        inv = modinv(Ab[pivot_row, col], p)
        Ab[pivot_row] = (Ab[pivot_row] * inv) % p

        # Eliminate other rows
        for row in range(m):
            if row != pivot_row and Ab[row, col] != 0:
                factor = Ab[row, col]
                Ab[row] = (Ab[row] - factor * Ab[pivot_row]) % p

        pivot_cols.append(col)
        pivot_row += 1
        if pivot_row == m:
            break

    # Check inconsistency
    for row in range(pivot_row, m):
        if np.all(Ab[row, :-1] == 0) and Ab[row, -1] != 0:
            return None  # no solution

    # Build one solution: set free vars = 0
    x = np.zeros(n, dtype=int)
    for i, col in enumerate(pivot_cols):
        row = i
        val = Ab[row, -1]
        for j in range(col + 1, n):
            val = (val - Ab[row, j] * x[j]) % p
        x[col] = val % p

    return x


In [27]:
A = np.array([[1,1,0],
              [0,1,1]])
b = np.array([1,2])

sols = solve_gfp(A, b, p=3)
for s in sols:
    print(s)


2
2
0


In [28]:
p=3
n=4

input_ps = np.random.randint(p, size=2 * n)
output_ps = np.random.randint(p, size=2 * n)

In [29]:
#def map_qudit_pauli_string_to_target():
# --- Input validation ---
if np.all(input_ps == 0) or np.all(output_ps == 0):
    raise ValueError("Vectors cannot be zero")
if not galois.is_prime(p):
    raise NotImplementedError(f"Prime dimension expected, got p={p}")

n= int(len(input_ps)//2)
a=symplectic_product(input_ps, output_ps,p)
print(a)

2


In [30]:
for p in range(2, 20):
    if galois.is_prime(p):
        for counts in range(3000):
            u = np.random.randint(p, size=2 * n)
            v = np.random.randint(p, size=2 * n)
            is_zero_case = np.array_equal(u, np.zeros(2 * n)) or np.array_equal(v, np.zeros(2 * n))
            if is_zero_case:
                # Zero vector case - correctly identified as impossible
                continue

            else:
                w= intermediate_transvection_solve(u, v, p)
                
                assert symplectic_product(u, w, p)!=0 and symplectic_product(w, v,p) !=0, f'{symplectic_product(u, w, p), symplectic_product(w, v, p)}'  


ValueError: Symplectic product must be zero

In [34]:
def intermediate_transvection_solve(u, v, p): 
    # solving in GF(p) instead of building for symplectic product = 0 case
    # --- Input validation ---
    if np.all(u == 0) or np.all(v == 0):
        raise ValueError("Input vectors cannot be zero")
    if not galois.is_prime(p):
        raise NotImplementedError(f"Prime dimension expected, got p={p}")
    

    if symplectic_product(u, v, p) !=0:
        w=(-u + v) % p
        #assert symplectic_product(u, w, p)!=0 and symplectic_product(w, v,p) !=0, f'{symplectic_product(u, w, p), symplectic_product(w, v, p)}'  
            
    else:
        #assert symplectic_product(u, v, p) ==0
        # Set up the linear system A @ w1 = b
        
        A = np.zeros((2, 2 * n), dtype=int)           # The structure of A is such that [ux|-uz
                                                                                        #  -vx|vz], 
                                                        #i.e, A @ w1 becomes [uxw1z- w1xuz
                                                        #                      w1xvz- w1zvx]

        w= np.zeros(2 * n, dtype=int)

        A[0, :n] = u[:n]  # X part of u multiplies Z part of w
        A[0, n:] = -u[n:]  # Z part of u multiplies X part of w


        A[1, :n] = -v[:n]  # X part of v multiplies Z part of w
        A[1, n:] = v[n:]  # Z part of v multiplies X part of w

        for p1, p2 in zip(range(1, p), range(1, p)):                                           
                b = np.array([p1, p2])                 #The structure of b is : [symplectic product(u, w, p)
                                                        #                         symplectic product(w, v, p)]  
                w1=solve_gfp(A, b, p)
                
                if w1 is not None:                       # The solution will be [w1z
                                                        #                        w1x], so, we will change it to w]   
                    w[:n]=w1[n:]
                    w[n:]= w1[:n]        
                    break

        #assert symplectic_product(u, w, p)!=0 and symplectic_product(w, v,p) !=0, f'{symplectic_product(u, w, p), symplectic_product(w, v, p)}'


    return w
    
    

In [32]:
def Find_transvection_map_solve(input_ps, output_ps, p):
    """
    Provides the map to transfer one PauliSum in GF(p) to another PauliSum.
    Currently works only for prime dimension. 
    Returns combined transvection (final map)
    """

    # --- Input validation ---
    if np.all(input_ps == 0) or np.all(output_ps == 0):
        raise ValueError("Vectors cannot be zero")
    if not galois.is_prime(p):
        raise NotImplementedError(f"Prime dimension expected, got p={p}")

    a= symplectic_product(input_ps, output_ps,p)
    if a != 0:
        ainv = modinv(a, p)
        h=(-input_ps + output_ps) % p
        F_h= transvection_matrix(h, p, multiplier=ainv)
        if (input_ps @ F_h % p != output_ps).all():
            raise ValueError("Failed to construct valid transvection for nonzero symplectic product")

    elif a == 0:
        w=intermediate_transvection_solve(input_ps, output_ps, p)
        a_w= symplectic_product(input_ps, w,p)
        a_w_inv = modinv(a_w, p)
        h=(-input_ps + w) % p
        F_h_1= transvection_matrix(h, p, multiplier=a_w_inv)
        if (input_ps @ F_h_1 % p != w).all():
            raise ValueError("Failed to construct valid transvection for u->w")
        b_w= symplectic_product(w, output_ps, p)
        b_w_inv = modinv(b_w, p)
        h=(-w + output_ps) % p
        F_h_2= transvection_matrix(h, p, multiplier=b_w_inv)
        if (w @ F_h_2 % p != output_ps).all():
            raise ValueError("Failed to construct valid transvection for w->v")
        F_h= F_h_1 @ F_h_2 %p
        if (input_ps @ F_h %p != output_ps).all():
            raise ValueError("Failed to construct valid transvection map: for u->v")

    return F_h

In [33]:
def map_pauli_sum_to_target_tableau(pauli_sum_tableau: np.ndarray, target_pauli_sum_tableau: np.ndarray) -> np.ndarray:
    """
    Map a Pauli sum to a target Pauli sum using symplectic transvections.
    """
    if not check_mappable_via_clifford(pauli_sum_tableau, target_pauli_sum_tableau):
        raise Exception(f'Cannot map\n{pauli_sum_tableau} to\n{target_pauli_sum_tableau}')
    # Find a vector w such that <w, pauli_sum> = <w, target_pauli_sum> = 1

    m = len(pauli_sum_tableau)
    n = len(pauli_sum_tableau[0]) // 2
    mapped_paulis = []
    F = np.eye(2 * n, dtype=int)
    for i in range(m):
        # update the starting point to whatever previous solutions mapped it to
        ps = (pauli_sum_tableau[i] @ F) % 2
        target_ps = target_pauli_sum_tableau[i]

        # if np.array_equal(ps, target_ps):
        #     continue

        F_map = map_single_pauli_string_to_target(ps, target_ps, mapped_paulis)
        assert np.all((ps @ F_map) % 2 == target_ps), f"\n{F_map}\n{ps}\n{(ps @ F_map) % 2}\n{target_ps}"
        for mp in mapped_paulis:
            assert np.all((mp @ F_map) % 2 == mp), f"\n{F_map}\n{mp}\n{(mp @ F_map) % 2}"
        mapped_paulis.append(target_ps)  # these are now the constraints for the next iteration
        F = (F @ F_map) % 2

    return F


In [34]:
for trial in range(100):
    n = 2
    allowed_dims = [2]  # , 3, 5, 7, 11]  # allowed dimensions
    dimensions = []  # dimensions
    for _ in range(n):
        dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
    m = 10 #num pauli
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]
    # scramble input hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab= pl_sum.tableau()
    output_tab= target_pl_sum.tableau()
    assert symplectic_product(input_tab[0], input_tab[1])== symplectic_product(output_tab[0], output_tab[1])
    
    p=2
    if check_mappable_via_clifford(input_tab, output_tab) and len(input_tab) > 1:
        F_h0= Find_transvection_map_solve_extended(input_tab[0], output_tab[0], constraints=[],sps=[], p=2)
        assert (input_tab[0] @ F_h0 %p == output_tab[0]).all()
        u2p = input_tab[1] @ F_h0 %p
        F_h1= Find_transvection_map_solve_extended(u2p, output_tab[1], constraints=[],sps=[], p=2)
        assert (u2p @ F_h1 %p == output_tab[1]).all()
        if symplectic_product(u2p, output_tab[1], p) !=0:
            assert (output_tab[0] @ F_h1 %p == output_tab[0]).all()
        else:
            sp_u1u2= symplectic_product(input_tab[0], input_tab[1], p)
            constraints= [output_tab[0]]
            sps= [sp_u1u2]
            F_h1= Find_transvection_map_solve_extended(u2p, output_tab[1], constraints=constraints,sps=sps, p=2)
            assert (u2p @ F_h1 %p == output_tab[1]).all() and (output_tab[0] @ F_h1 %p == output_tab[0]).all()
    #print(trial)

In [ ]:
def intermediate_transvection_solve_extended(u, v, constraints, sps,  p): 
    
    # solving in GF(p) instead of building for symplectic product = 0 case
    # --- Input validation ---
    if np.all(u == 0) or np.all(v == 0):
        raise ValueError("Input vectors cannot be zero")
    if np.all(u==v):
         raise ValueError("Input PauliSum == Output PauliSum")

    if not galois.is_prime(p):
        raise NotImplementedError(f"Prime dimension expected, got p={p}")
    
    if symplectic_product(u, v, p) != 0:
        raise ValueError("Symplectic product must be zero")

    
            
    else:
        t= len(constraints)
        n= int(len(u)//2)
        # Set up the linear system A @ w1 = b
        
        A = np.zeros((2+t, 2 * n), dtype=int)           # The structure of A is such that [ux|-uz
                                                                                        #  -vx|vz], 
                                                        #i.e, A @ w1 becomes [uxw1z- w1xuz
                                                        #                      w1xvz- w1zvx]

        w= np.zeros(2 * n, dtype=int)

        A[0, :n] = u[:n]  # X part of u multiplies Z part of w
        A[0, n:] = -u[n:]  # Z part of u multiplies X part of w


        A[1, :n] = -v[:n]  # X part of v multiplies Z part of w
        A[1, n:] = v[n:]  # Z part of v multiplies X part of w

        for el in range(t):
             A[2+el, :n] = constraints[el][:n]
             A[2+el, n:]= -constraints[el][n:]

        b= np.zeros(2+t, dtype=int)
        for el in range(t):
             b[2+el]= sps[el]
             
        for p1 in range(1, p):
            for p2 in range(1, p):                                           
                    b[0] = p1              #The structure of b is : [symplectic product(u, w, p)
                    b[1] = p2                                     #                         symplectic product(w, v, p)]  
                    w1=solve_gfp(A, b, p)
                    #print(p1, p2, w1)
                    if w1 is not None:
                            break
            break
        #print(p1, p2, w1)                       # The solution will be [w1z                                   #                        w1x], so, we will change it to w]   
        w[:n]=w1[n:]
        w[n:]= w1[:n] 

        if np.all(w==0):
             raise ValueError('No solns found')
        else:
            assert symplectic_product(u, w, p)!=0 and symplectic_product(w, v,p) !=0, f'{symplectic_product(u, w, p), symplectic_product(w, v, p)}'


    return w

In [120]:
def Find_transvection_map_solve_extended(input_ps, output_ps,constraints=[], sps=[], p=2):
    """
    Provides the map to transfer one PauliString in GF(p) to another PauliString, subject to some constraints.
    Currently works only for prime dimension. 
    Returns combined transvection (final map)
    """

    # --- Input validation ---
    if np.all(input_ps == 0) or np.all(output_ps == 0):
        raise ValueError("Vectors cannot be zero")
    if not galois.is_prime(p):
        raise NotImplementedError(f"Prime dimension expected, got p={p}")

    a= symplectic_product(input_ps, output_ps,p)
    if a != 0:
        ainv = modinv(a, p)
        h=(-input_ps + output_ps) % p
        F_h= transvection_matrix(h, p, multiplier=ainv)
        if (input_ps @ F_h % p != output_ps).all():
            raise ValueError("Failed to construct valid transvection for nonzero symplectic product")

    elif a == 0:
        if np.all(input_ps==output_ps):
            F_h= np.eye(2 * n, dtype=int)
        else:
            w=intermediate_transvection_solve_extended(input_ps, output_ps, constraints=constraints, sps=sps, p=p)
            a_w= symplectic_product(input_ps, w,p)
            a_w_inv = modinv(a_w, p)
            h=(-input_ps + w) % p
            F_h_1= transvection_matrix(h, p, multiplier=a_w_inv)
            if (input_ps @ F_h_1 % p != w).all():
                raise ValueError("Failed to construct valid transvection for u->w")
            b_w= symplectic_product(w, output_ps, p)
            b_w_inv = modinv(b_w, p)
            h=(-w + output_ps) % p
            F_h_2= transvection_matrix(h, p, multiplier=b_w_inv)
            if (w @ F_h_2 % p != output_ps).all():
                raise ValueError("Failed to construct valid transvection for w->v")
            F_h= F_h_1 @ F_h_2 %p
            if (input_ps @ F_h %p != output_ps).all():
                raise ValueError("Failed to construct valid transvection map: for u->v")

    return F_h

In [121]:
for trial in range(100):
    n = 2
    allowed_dims = [3]  # , 3, 5, 7, 11]  # allowed dimensions
    dimensions = [3]*n  # dimensions
    for _ in range(n):
        dimensions.append(int(np.random.choice(allowed_dims))) # , 3, 5, 7, 11]  # allowed dimensions
    m = 10 #num pauli
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)
    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]
    # scramble input hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)
    p=3
    input_tab= pl_sum.tableau()
    output_tab= target_pl_sum.tableau()
    if symplectic_product(input_tab[0], input_tab[1], p) != symplectic_product(output_tab[0], output_tab[1], p):
        continue

In [122]:
symplectic_product(input_tab[0], input_tab[1], p), symplectic_product(output_tab[0], output_tab[1], p)

(np.int64(1), np.int64(1))

In [123]:

    if check_mappable_via_clifford(input_tab, output_tab) and len(input_tab) > 1:
        constraints=[]
        sps=[]
        for lt in range(len(input_tab)):
            F_h0= Find_transvection_map_solve(input_tab[lt], output_tab[lt], p)
            assert (input_tab[lt] @ F_h0 %p == output_tab[lt]).all()
            u2p = input_tab[1] @ F_h0 %p
            F_h1= Find_transvection_map_solve(u2p, output_tab[1], p)
            assert (u2p @ F_h1 %p == output_tab[1]).all()
            if symplectic_product(u2p, output_tab[1], p) !=0:
                assert (output_tab[0] @ F_h1 %p == output_tab[0]).all()
            else:
                sp_u1u2= symplectic_product(input_tab[0], input_tab[1], p)
                constraints.append(output_tab[0])
                sps.append(sp_u1u2)
                w= intermediate_transvection_solve_extended(u2p, output_tab[1], constraints,sps, p)
    #print(trial)

In [124]:
for _ in range(100):
    n = 2  # number of subsystems stays fixed
    allowed_dims = [2]  # can extend to [2, 3, 5, ...] if needed
    dimensions = [int(np.random.choice(allowed_dims)) for _ in range(n)]

    m = 10  # number of Paulis to test (now extendable beyond 2!)
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)

    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]

    # scramble input Hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab = pl_sum.tableau()
    output_tab = target_pl_sum.tableau()

    p = 2
    if check_mappable_via_clifford(input_tab, output_tab):

        F_total= map_paulisum_to_target_paulisum(input_tab, output_tab, p)

        assert (input_tab @ F_total % p == output_tab).all()


1 1 [1 1 0 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [1 0 0 0]
1 1 [0 0 1 1]
1 1 [0 1 0 0]
1 1 [1 0 0 1]
1 1 [0 0 0 1]
1 1 [1 0 0 0]
1 1 [1 0 1 0]
1 1 [0 1 0 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [1 1 0 1]
1 1 [1 0 0 0]
1 1 [1 0 1 0]
1 1 [0 1 0 0]
1 1 [0 0 1 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [1 1 1 0]
1 1 [1 0 0 0]
1 1 [1 0 1 0]
1 1 [0 0 1 0]
1 1 [0 1 0 0]
1 1 [0 1 0 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [0 1 0 0]
1 1 [0 1 0 0]
1 1 [1 0 0 1]
1 1 [0 1 0 0]
1 1 [1 0 0 0]
1 1 [1 1 0 1]
1 1 [0 1 0 0]
1 1 [1 1 0 0]
1 1 [0 0 0 1]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [0 1 1 0]
1 1 [1 0 0 0]
1 1 [1 0 1 0]
1 1 [1 0 0 0]
1 1 [1 1 0 1]
1 1 [0 1 0 0]
1 1 [1 0 1 0]
1 1 [1 0 0 0]
1 1 [1 0 0 1]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [1 0 0 1]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [0 0 1 0]
1 1 [0 0 1 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [0 0 1 0]
1 1 [1 0 0 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [1 0 0 0]
1 1 [0 1 0 0]
1 1 [0 1 0 0]
1 1 [0 0 1 0]
1 1 [0 1 1 0]
1 1 [0 1 0 0]
1 1 [1

In [56]:
def map_paulisum_to_target_paulisum(input_tab, output_tab, p):

    if not check_mappable_via_clifford(input_tab, output_tab,p):
        raise Exception(f'Cannot map these tabs')

    # Map the first Pauli
    F_total = Find_transvection_map_solve_extended(
        input_tab[0], output_tab[0], constraints=[], sps=[], p=p
    )
    assert (input_tab[0] @ F_total % p == output_tab[0]).all()

    # Iteratively map the rest of the Paulis
    for k in range(1, len(input_tab)):
        u_k = input_tab[k] @ F_total % p

        # Constraints: previously mapped Paulis must stay fixed
        constraints = [output_tab[j] for j in range(k)]
        sps = [symplectic_product(input_tab[j], input_tab[k], p) for j in range(k)]

        F_k = Find_transvection_map_solve_extended(
            u_k, output_tab[k], constraints=constraints, sps=sps, p=p
        )

        # Update cumulative Clifford map
        F_total = (F_total @ F_k) % p

        # Verify this step
        assert (u_k @ F_k % p == output_tab[k]).all()
        for j in range(k):
            assert (output_tab[j] @ F_k % p == output_tab[j]).all()

    return F_total

In [57]:
input_tab

array([[1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 0, 1],
       [1, 0, 1, 1]])

In [58]:
input_tab @ F_total % p

array([[1, 0, 1, 0],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0]])

In [59]:
output_tab

array([[1, 0, 1, 0],
       [1, 1, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0]])

In [60]:
n=2
p=3
m=10
allowed_dims = [3]  # can extend to [2, 3, 5, ...] if needed
dimensions = [3]*n
for _ in range(100):
    pl_sum = random_hamiltonian.random_pauli_hamiltonian(m, dimensions)

    basis_indices, _ = get_linear_dependencies(pl_sum.tableau(), int(pl_sum.lcm))
    pl_sum = pl_sum[basis_indices]

    # scramble input Hamiltonian to get target
    C = Circuit.from_random(len(dimensions), 10 * n**2, dimensions=dimensions)
    target_pl_sum = C.act(pl_sum)

    input_tab = pl_sum.tableau()
    output_tab = target_pl_sum.tableau()

    if check_mappable_via_clifford(input_tab, output_tab, p):

        F_total= map_paulisum_to_target_paulisum(input_tab, output_tab, p)

        assert (input_tab @ F_total % p == output_tab).all(), f'could not map for {input_tab, output_tab}'


AssertionError: (np.int64(0), np.int64(0))

In [61]:
input_tab

array([[2, 2, 2, 1],
       [2, 2, 1, 2],
       [2, 1, 1, 2],
       [2, 1, 1, 0]])

In [62]:
output_tab

array([[2, 2, 1, 2],
       [2, 1, 0, 2],
       [2, 2, 1, 1],
       [2, 1, 1, 1]])

In [63]:
if not check_mappable_via_clifford(input_tab, output_tab,p):
    raise Exception(f'Cannot map these tabs')

# Map the first Pauli
F_total = Find_transvection_map_solve_extended(
    input_tab[0], output_tab[0], constraints=[], sps=[], p=p
)
assert (input_tab[0] @ F_total % p == output_tab[0]).all()
print(F_total)
# Iteratively map the rest of the Paulis
for k in range(1, len(input_tab)):
    u_k = input_tab[k] @ F_total % p

    # Constraints: previously mapped Paulis must stay fixed
    constraints = [output_tab[j] for j in range(k)]
    sps = [symplectic_product(input_tab[j], input_tab[k], p) for j in range(k)]
    print()
    print(k, F_total, input_tab[k],u_k, output_tab[k])
    F_k = Find_transvection_map_solve_extended(
        u_k, output_tab[k], constraints=constraints, sps=sps, p=p
    )

    # Update cumulative Clifford map
    F_total = (F_total @ F_k) % p

    # Verify this step
    assert (u_k @ F_k % p == output_tab[k]).all()
    for j in range(k):
        assert (output_tab[j] @ F_k % p == output_tab[j]).all()

[[0 1 1 2]
 [1 0 2 1]
 [0 0 0 1]
 [0 0 1 0]]

1 [[0 1 1 2]
 [1 0 2 1]
 [0 0 0 1]
 [0 0 1 0]] [2 2 1 2] [2 2 2 1] [2 1 0 2]

2 [[0 1 1 2]
 [1 2 0 2]
 [0 1 2 0]
 [0 0 1 0]] [2 1 1 2] [1 2 0 0] [2 2 1 1]


AssertionError: (np.int64(0), np.int64(0))

In [9]:
F_total = Find_transvection_map_solve_extended(
    input_tab[0], output_tab[0], constraints=[], sps=[], p=p)
assert (input_tab[0] @ F_total % p == output_tab[0]).all()

In [12]:
input_tab[0] @ F_total % p, output_tab[0]

(array([1, 0, 1, 1]), array([1, 0, 1, 1]))

In [ ]:
for k in range(1, len(input_tab)):
    u_k = input_tab[k] @ F_total % p

    # Constraints: previously mapped Paulis must stay fixed
    constraints = [output_tab[j] for j in range(k)]
    sps = [symplectic_product(input_tab[j], input_tab[k], p) for j in range(k)]

    F_k = Find_transvection_map_solve_extended(
        u_k, output_tab[k], constraints=constraints, sps=sps, p=p
    )
 


AssertionError: (np.int64(0), np.int64(0))

In [67]:
print(u_k, output_tab[k], constraints, sps, p)

[1 2 0 0] [2 2 1 1] [array([2, 2, 1, 2]), array([2, 1, 0, 2])] [np.int64(1), np.int64(2)] 3


In [69]:
input_ps, output_ps= u_k, output_tab[k]

In [70]:
a= symplectic_product(input_ps, output_ps,p)
if a != 0:
    ainv = modinv(a, p)
    h=(-input_ps + output_ps) % p
    F_h= transvection_matrix(h, p, multiplier=ainv)
    if (input_ps @ F_h % p != output_ps).all():
        raise ValueError("Failed to construct valid transvection for nonzero symplectic product")

elif a == 0:
    w=intermediate_transvection_solve_extended(input_ps, output_ps, constraints=constraints, sps=sps, p=p)
    a_w= symplectic_product(input_ps, w,p)
    a_w_inv = modinv(a_w, p)
    h=(-input_ps + w) % p
    F_h_1= transvection_matrix(h, p, multiplier=a_w_inv)
    if (input_ps @ F_h_1 % p != w).all():
        raise ValueError("Failed to construct valid transvection for u->w")
    b_w= symplectic_product(w, output_ps, p)
    b_w_inv = modinv(b_w, p)
    h=(-w + output_ps) % p
    F_h_2= transvection_matrix(h, p, multiplier=b_w_inv)
    if (w @ F_h_2 % p != output_ps).all():
        raise ValueError("Failed to construct valid transvection for w->v")
    F_h= F_h_1 @ F_h_2 %p
    if (input_ps @ F_h %p != output_ps).all():
        raise ValueError("Failed to construct valid transvection map: for u->v")

AssertionError: (np.int64(0), np.int64(0))

In [38]:
w=intermediate_transvection_solve_extended(input_ps, output_ps, constraints=constraints, sps=sps, p=p)

(array([1, 2, 0, 0]),
 array([2, 2, 1, 1]),
 [array([2, 2, 1, 2]), array([2, 1, 0, 2])],
 [np.int64(1), np.int64(2)],
 3)

In [74]:
u, v, constraints, sps, p=input_ps, output_ps, constraints, sps, p

In [118]:
t= len(constraints)
n= int(len(u)//2)
# Set up the linear system A @ w1 = b

A = np.zeros((2+t, 2 * n), dtype=int)           # The structure of A is such that [ux|-uz
                                                                                #  -vx|vz], 
                                                #i.e, A @ w1 becomes [uxw1z- w1xuz
                                                #                      w1xvz- w1zvx]

w= np.zeros(2 * n, dtype=int)

A[0, :n] = u[:n]  # X part of u multiplies Z part of w
A[0, n:] = -u[n:]  # Z part of u multiplies X part of w


A[1, :n] = -v[:n]  # X part of v multiplies Z part of w
A[1, n:] = v[n:]  # Z part of v multiplies X part of w

for el in range(t):
        A[2+el, :n] = constraints[el][:n]
        A[2+el, n:]= -constraints[el][n:]

b= np.zeros(2+t, dtype=int)
for el in range(t):
        b[2+el]= sps[el]
        
for p1 in range(1, p):
        for p2 in range(1, p):                                           
                b[0] = p1              #The structure of b is : [symplectic product(u, w, p)
                b[1] = p2                                     #                         symplectic product(w, v, p)]  
                w1=solve_gfp(A, b, p)
                #print(p1, p2, w1)
                if w1 is not None:
                        break
        break
print(p1, p2, w1)                       # The solution will be [w1z                                   #                        w1x], so, we will change it to w]   
w[:n]=w1[n:]
w[n:]= w1[:n] 
               

1 2 [0 2 0 0]


In [102]:
w

array([0, 0, 0, 0])

In [103]:
A

array([[ 1,  2,  0,  0],
       [-2, -2,  1,  1],
       [ 2,  2, -1, -2],
       [ 2,  1,  0, -2]])

In [104]:
u, v, constraints, sps, p

(array([1, 2, 0, 0]),
 array([2, 2, 1, 1]),
 [array([2, 2, 1, 2]), array([2, 1, 0, 2])],
 [np.int64(1), np.int64(2)],
 3)

In [100]:
b

array([1, 2, 1, 2])